In [1]:
#Import libraries
import os 
from typing import Annotated
from openai import AsyncOpenAI

In [2]:
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function

In [3]:
import random   

# Define a sample plugin for the sample

class DestinationsPlugin:
    """A List of Random Destinations for a vacation."""

    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # Track last destination to avoid repeats
        self.last_destination = None

    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]:
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # Select a random destination
        destination = random.choice(available_destinations)

        # Update the last destination
        self.last_destination = destination

        return destination

In [15]:
client = AsyncOpenAI(
    api_key="your_api_key", 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

In [16]:
#Create the Agent
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions="You are a helpful AI Agent that can help plan vacations for customers at random destinations",
)

In [17]:
#Run the Agent
async def main():
    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Plan me a day trip.",
    ]

    for user_input in user_inputs:
        print(f"# User: {user_input}\n")
        first_chunk = True
        async for response in agent.invoke_stream(
            messages=user_input, thread=thread,
        ):
            # 5. Print the response
            if first_chunk:
                print(f"# {response.name}: ", end="", flush=True)
                first_chunk = False
            print(f"{response}", end="", flush=True)
            thread = response.thread
        print()

    # Clean up the thread
    await thread.delete() if thread else None

await main()

# User: Plan me a day trip.

# TravelAgent: How about a day trip to Barcelona, Spain? Here’s a suggested itinerary for you:

### Morning
- **Breakfast at a Local Café**: Start your day with a traditional Spanish breakfast at a local café. Try "pan con tomate" (bread with tomato) and a coffee.
- **Visit Park Güell**: Explore this vibrant park designed by Antoni Gaudí, filled with beautiful mosaics, gardens, and stunning views of the city.

### Midday
- **Lunch at Mercat de Sant Josep de la Boqueria**: Head to La Boqueria market for lunch. Enjoy fresh tapas and the local delicacies from various stalls.
- **Walk along La Rambla**: Stroll down this famous street, where you can do some shopping, enjoy street performances, and soak in the lively atmosphere.

### Afternoon
- **Sagrada Familia**: Visit the iconic basilica. Make sure to book tickets in advance to skip the lines.
- **Explore the Gothic Quarter**: Wander through the narrow streets, visit the Barcelona Cathedral, and discover uniq